# Get Service Providers

This notebook is used to grab service provider information from JustWatch. The only way to search using this API is by title, so the results are imperfect. A fuzzywuzzy ratio is currently calculated to try to determine if the search result is a good match. 

In [1]:
# imports 
from justwatch import JustWatch # https://github.com/dawoudt/JustWatchAPI
import psycopg2
from multiprocessing import Pool
from tqdm._tqdm_notebook import tqdm_notebook
import time
from fuzzywuzzy import fuzz # https://github.com/seatgeek/fuzzywuzzy
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
# load .env file for keys
load_dotenv()

In [2]:
# make justwatch instance
just_watch = JustWatch(country='US')

In [3]:
# make providers list to narrow search
provider_details = just_watch.get_providers()
my_providers = []
other_providers = ['nfx', 'amp', 'hlu', 'amz', 'dnp', 'yot']

for provider in provider_details:
    if len(provider['data']['deeplink_data']) > 0:
        my_providers.append((
            provider['id'], 
            provider['short_name'], 
            provider['clear_name'], 
            provider['data']['deeplink_data'][0]['path_template']
        ))

my_providers[0], len(my_providers)

((8, 'nfx', 'Netflix', 'www.netflix.com/watch/%DEEPLINK%'), 42)

In [4]:
# test API
result = just_watch.search_for_item(query='the matrix')
if result['total_results'] > 0:
    test_item = result['items'][0]
test_item.keys()

dict_keys(['jw_entity_id', 'id', 'title', 'full_path', 'full_paths', 'poster', 'original_release_year', 'tmdb_popularity', 'object_type', 'localized_release_date', 'offers', 'scoring', 'cinema_release_date'])

In [5]:
# test get by id
the_matrix = just_watch.get_title(title_id=test_item['id'])
the_matrix['title']

'The Matrix'

In [6]:
test_item['id'], test_item['title'], test_item['offers'][:2]

(10,
 'The Matrix',
 [{'monetization_type': 'rent',
   'provider_id': 7,
   'retail_price': 3.99,
   'currency': 'USD',
   'urls': {'standard_web': 'https://www.vudu.com/content/movies/details/The-Matrix/9254'},
   'presentation_type': 'sd',
   'date_provider_id': '2000-01-01_7',
   'date_created': '2000-01-01'},
  {'monetization_type': 'buy',
   'provider_id': 7,
   'retail_price': 9.99,
   'last_change_retail_price': 4.99,
   'last_change_difference': 5,
   'last_change_percent': 100.2004,
   'last_change_date': '2020-04-08',
   'last_change_date_provider_id': '2020-04-08_7',
   'currency': 'USD',
   'urls': {'standard_web': 'https://www.vudu.com/content/movies/details/The-Matrix/9254'},
   'presentation_type': 'sd',
   'date_provider_id': '2000-01-01_7',
   'date_created': '2000-01-01'}])

In [4]:
# create connection to prod DB
connection = psycopg2.connect(
                            user=os.getenv('db_user'),
                            password=os.getenv('db_password'),
                            host=os.getenv('db_host'),
                            port="5432",
                            database=os.getenv('db_user')
                            )

In [19]:
# get all movies
cursor = connection.cursor()
postgreSQL_select_Query = "SELECT movie_id, primary_title FROM imdb_movies;"
cursor.execute(postgreSQL_select_Query)
all_movies = cursor.fetchall()
len(all_movies)

301967

In [6]:
# function to search for movie
"""
Now saving a ratio score from fuzzywuzzy 
to later determine if the result is the correct movie
Could improve by calculating ratio after collecting JustWatch info
because currently the collection process is slow (~1.5hr per 50k)

I'm finding it may be better to broaden the providers in the 
search to more effectively get the correct movie, then can 
filter down to the offer data from the providers we want.
"""
def get_movie(movie_data, providers=['nfx', 'amp', 'hlu']):
    movie_id, title = movie_data
    # only other way I see to improve this query is with:
    # release_year_from and release_year_until
    try:
        # could also save original_release_year for verifying match 
        # and poster may be another source for posters
        # because they also provide tmdb_popularity I'm guessing id may be linked to tmdb
        result = just_watch.search_for_item(query=title, providers=providers)
        result = result['items'][0]
        return {
            'movie_id': movie_id,
            'movie_title': title,
            'id': result['id'],
            'title': result['title'], 
            'offers': result['offers'],
            'ratio': fuzz.ratio(result['title'].lower(),title.lower())
        }
    except: 
        return None

In [10]:
# get movie by id
def get_by_id(title_id):
    return just_watch.get_title(title_id=title_id)

In [11]:
get_movie(all_movies[100000]), all_movies[100000][1]

({'movie_id': '0118901',
  'movie_title': 'Critical Care',
  'id': 14597,
  'title': 'The Care Bears',
  'offers': [{'type': 'aggregated',
    'monetization_type': 'flatrate',
    'provider_id': 8,
    'currency': 'USD',
    'urls': {'standard_web': 'http://www.netflix.com/title/70287604',
     'deeplink_android_tv': '70282094',
     'deeplink_fire_tv': 'intent://www.netflix.com/watch/70282094#Intent;launchFlags=0x00800000;scheme=https;package=com.netflix.ninja;S.source=30;end',
     'deeplink_tvos': 'nflx://www.netflix.com/watch/70287604'},
    'presentation_type': 'sd',
    'element_count': 1,
    'new_element_count': 1,
    'date_provider_id': '2019-08-02_8',
    'date_created': '2019-08-02'},
   {'type': 'aggregated',
    'monetization_type': 'flatrate',
    'provider_id': 8,
    'currency': 'USD',
    'urls': {'standard_web': 'http://www.netflix.com/title/70287604',
     'deeplink_android_tv': '70282094',
     'deeplink_fire_tv': 'intent://www.netflix.com/watch/70282094#Intent;lau

In [24]:
# test func on sample
movie_sample = all_movies[250000:]
start = time.process_time()
with Pool(5) as p:
    provider_data = list(tqdm_notebook(
                p.imap(get_movie, movie_sample),
                total=len(movie_sample)
                ))
time.process_time() - start

104.58164

In [25]:
# check results of collection
provider_count = 0
movie_ids = []
jw_ids = []
titles = []
jw_titles = []
offer_ids = []
offer_urls = []
ratios = []

for provider in provider_data:
    if provider is not None:
        provider_count += 1
        movie_ids.append(provider['movie_id'])
        jw_ids.append(provider['id'])
        titles.append(provider['movie_title'])
        jw_titles.append(provider['title'])
        # other offer info: presentation_type (sd, hd), 
        # date_created, monetization_type (flat_rate, buy, rent)
        offer_ids.append(",".join([str(prov['provider_id']) for prov in provider['offers']]))
        offer_urls.append(",".join([prov['urls']['standard_web'] for prov in provider['offers'] if 'standard_web' in prov['urls']]))
        ratios.append(provider['ratio'])
        
str(round(provider_count / len(provider_data), 2)*100)+"%"

'88.0%'

In [26]:
# make df with results
prov_df = pd.DataFrame({
    'movie_id': movie_ids,
    'title': titles,
    'jw_id': jw_ids,
    'jw_title': jw_titles,
    'offer_provider_id': offer_ids,
    'offer_urls': offer_urls,
    'ratio': ratios
})
print(prov_df.shape)
prov_df.head()

(45987, 7)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
0,0288034,Gottes zweite Garnitur,366889,Vice,"68,68,3,192,3,192,68,68,10,10,10,10,2,2,2,2,7,...",https://www.microsoft.com/en-us/p/vice/8d6kgwx...,15
1,0288035,The Way to the Sun,98815,The Way Back,"87,3,192,192,3,212,68,68,68,68,2,2,2,2,7,7,7,7...","https://acorn.tv/wayback/,https://play.google....",53
2,0288040,Hatred of a Minute,353120,I'm Dreaming of a White Doomsday,"10,10,10,10,331,331,331,331,331,9,9",https://www.amazon.com/gp/product/B07YSX1QSV?c...,44
3,0288042,The Heart Is a Rebel,184664,Roland Klick: The Heart Is a Hungry Hunter,"10,10,10,10,9,9,2,2,2,2",https://www.amazon.com/gp/product/B07S7R5K3Z?c...,55
4,0288054,"Honey, My Love, So Sweet",223380,"BOFURI: I Don’t Want to Get Hurt, so I’ll Max ...","10,10,3,3,15,15,269,269",https://www.amazon.com/gp/product/B0845T4KNR?c...,31


In [27]:
# save prov_df
prov_df.to_csv('provider_data_250k_on.csv', index=False)

In [29]:
# concat all provider_data_*.csv files
df_1 = pd.read_csv('provider_data_0_50k.csv')
df_2 = pd.read_csv('provider_data_50k_100k.csv')
df_3 = pd.read_csv('provider_data_100k_150k.csv')
df_4 = pd.read_csv('provider_data_150k_200k.csv')
df_5 = pd.read_csv('provider_data_250k_on.csv')

prov_df = pd.concat([df_1, df_2, df_3, df_4, df_5])
print(prov_df.shape)
prov_df.head()

(224382, 7)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
0,1051262,The Stalker Within,309961,The Evil Within,"7,7,7,7,192,3,3,192,3,192,9,9,10,10,10,10,3,19...",https://www.vudu.com/content/movies/details/Th...,73
1,1051263,Stolen Life,167298,Stolen Life,"25,10,10,9","https://www.fandor.com/films/stolen_life,https...",100
2,10513072,Don't Date Your Sister,434938,Don't Open Your Eyes,"10,10,10,10,2,2,2,2,12,9,9",https://www.amazon.com/gp/product/B07H4T4NQ2?c...,67
3,1051320,La cantatrice chauve,23259,La Bamba,"7,7,7,7,3,3,192,192,279,279,279,279,10,10,10,1...",https://www.vudu.com/content/movies/details/La...,36
4,10513286,Historia de mi nombre,403709,Marriage Story,"8,8,8","http://www.netflix.com/title/80223779,http://w...",40


In [30]:
# save concatenated df
prov_df.to_csv('provider_data.csv', index=False)

In [31]:
# checking stats of ratios to get a guess on how many matches were found
prov_df['ratio'].describe()

count    224382.000000
mean         51.437958
std          25.091076
min           0.000000
25%          32.000000
50%          47.000000
75%          67.000000
max         100.000000
Name: ratio, dtype: float64

In [32]:
# filter down to rows with ratio > 90
filtered_df = prov_df[prov_df['ratio'] > 90]
print(filtered_df.shape)
filtered_df.head()

(29017, 7)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
1,1051263,Stolen Life,167298,Stolen Life,"25,10,10,9","https://www.fandor.com/films/stolen_life,https...",100
9,10511068,Inheritance,311393,Inheritance,"3,3,7,7,7,7,192,192,3,3,192,192,9,9,10,10,10,10",https://play.google.com/store/movies/details/I...,100
11,10514932,What Do I Do Now?,659810,What Do I Do Now?,"9,9,10,10,10,10",https://www.amazon.com/gp/product/B075QVYP3H?c...,100
28,1051245,Moving Midway,78930,Moving Midway,"2,2,191,9,9",https://itunes.apple.com/us/movie/moving-midwa...,100
31,1051253,The Portal,319476,The Portal,"3,3,3,10,10,10,10,9,9,192,192,192,2,2,2,2,68,6...",https://play.google.com/store/movies/details/T...,100


In [33]:
# filtered_df looks much better, lets check the worst of them
filtered_df.sort_values(by='ratio')[:10]

,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
34760,247931,Antham,419091,Antam,"218,218,9,9",https://erosnow.com/movie/watch/1065669/Antham...,91
38962,85900,Le marginal,42022,El marginal,"8,8","http://www.netflix.com/title/80115297,http://w...",91
22463,4344742,Women and Sometimes Men,418028,Women & Sometimes Men,"191,2,2,2,2,10,10,10,10,3,192,3,3,192,192,212,...",https://www.kanopy.com/product/women-and-somet...,91
31613,1954796,Sahar,270987,Sahara,"8,8","http://www.netflix.com/title/80161029,http://w...",91
20379,4150196,U.F.O,60061,U.F.O.,"192,192,10,10,10,10,9,9,105,105,105,105","https://www.youtube.com/watch?v=BZ7VuxArTY4,ht...",91
31560,1954381,Dhada,371764,Dhadak,9,https://www.amazon.com/gp/product/B07HKC1PSZ?c...,91
33348,4793322,Defenders of the Faith,9543,Defenders of the Earth,"9,9,10,241,300",https://www.amazon.com/gp/product/B06X9P2R5V?c...,91
39208,86193,Richard III,249063,Richard III,"9,9",https://www.amazon.com/gp/product/B07TYJBC15?c...,91
13416,376296,Tramp,244134,Tramps,"8,8","http://www.netflix.com/title/80146759,http://w...",91
24724,6459680,Danga,245671,Dangal,"8,8","http://www.netflix.com/title/80166185,http://w...",91


## Retrospective:

`filtered_df` is looking good, but it may be cool to incorperate a user-feedback feature with these links where they vote if they are correct or not. (more advanced version would be to save a couple from each search and have the user pick which one is correct if any)